## Import de Librerias

In [90]:
import face_recognition
from extraccion_de_caracteristicas import load_json
import time
import os 


In [91]:
from knn_secuencial import range_search #Pronto integrar aca
from knn_rtree import * #Pronto corregir

## Carga de los encodings del Dataset

In [70]:
dataset = load_json()

## Pruebas

In [113]:
N_values = [100,200,400,800,1600,3200,6400,12800]
k = 8   # fijo
image_path = "./test/teofilo.png"
query_image = face_recognition.load_image_file(image_path)
faces_encoding = face_recognition.face_encodings(query_image)[0]

/home/diego/anaconda3/envs/mainds/lib/python3.9/site-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


### KNN Sequential

In [110]:
import face_recognition
import heapq as pq

In [114]:
def knn_pq(faces_encoding, dataset, k):
    result = []
    for path, matrix_vector_faces in dataset:
        for distance in face_recognition.face_distance(matrix_vector_faces, faces_encoding):
            #result.append((os.path.basename(path), distance))
            pq.heappush(result, (distance, os.path.basename(path)))

    resultK = pq.nsmallest(k , result)

    return [os.path.basename(path) for distance, path in resultK]


In [146]:
''' Testeando KNN-Secuencial PQ'''
print("KNN-Secuencial PQ TEST")
for n in N_values:
    print("N =",n)
    start_time = time.time()
    data = dataset[:n]
    knn_PQ = knn_pq(faces_encoding, data, k)
    print(knn_PQ)
    end_time = time.time()
    tiempo_ejecucion = end_time - start_time
    print("El tiempo de exec es: ", tiempo_ejecucion)

KNN-Secuencial PQ TEST
N = 100
['Angel_Maza_0001.jpg', 'Anthony_Pico_0001.jpg', 'Gro_Harlem_Brundtland_0001.jpg', 'Reginald_Hudlin_0001.jpg', 'Mike_Smith_0001.jpg', 'Xiang_Huaicheng_0001.jpg', 'Dragan_Covic_0001.jpg', 'Ranil_Wickremasinghe_0001.jpg']
El tiempo de exec es:  0.005017280578613281
N = 200
['Wilbert_Elki_Meza_Majino_0001.jpg', 'Jeong_Se-hyun_0001.jpg', 'Jeong_Se-hyun_0004.jpg', 'Jeong_Se-hyun_0009.jpg', 'Jeong_Se-hyun_0003.jpg', 'Jeong_Se-hyun_0006.jpg', 'Angel_Maza_0001.jpg', 'Jeong_Se-hyun_0008.jpg']
El tiempo de exec es:  0.016119003295898438
N = 400
['George_Lopez_0003.jpg', 'Wilbert_Elki_Meza_Majino_0001.jpg', 'Jeong_Se-hyun_0001.jpg', 'Jeong_Se-hyun_0004.jpg', 'Jeong_Se-hyun_0009.jpg', 'Jeong_Se-hyun_0003.jpg', 'Jeong_Se-hyun_0006.jpg', 'George_Lopez_0005.jpg']
El tiempo de exec es:  0.014310836791992188
N = 800
['George_Lopez_0003.jpg', 'Miguel_Estrada_0001.jpg', 'Gary_Locke_0001.jpg', 'Chen_Liang_Yu_0002.jpg', 'Wilbert_Elki_Meza_Majino_0001.jpg', 'Miguel_Estrada_000

### KNN Range Search

In [148]:
def select_representative_radio(mean_distance, std_distance):
    # regla empírica
    r1 = mean_distance - std_distance
    r2 = mean_distance - 2 * std_distance
    r3 = mean_distance - 3 * std_distance

    return [r1, r2, r3]

def distances(dataset):
    vector_dist = []
    for path, matrix_vector_faces in dataset:
        for distance in face_recognition.face_distance(matrix_vector_faces, faces_encoding):
            vector_dist.append(distance)

    return list((np.mean(vector_dist), np.std(vector_dist)))

distances = distances(dataset)
radios = select_representative_radio(distances[0],distances[1])
print("Estos son los radios para probar de manera representativa segun la distribucion de la data")
print(radios)


Estos son los radios para probar de manera representativa segun la distribucion de la data
[0.709143092728737, 0.6196988861402919, 0.5302546795518467]


In [149]:
def range_search(faces_encoding, dataset, radio):
    result = []
    for path, matrix_vector_faces in dataset:
        for distance in face_recognition.face_distance(matrix_vector_faces, faces_encoding):
            if distance < radio:
                result.append(( os.path.basename(path), distance))

    if len(result):
        result = sorted(result, key = lambda x: x[1])
        return [path for path, dis in result]

In [150]:
''' Testeando KNN-Secuencial RS '''
print("KNN-Secuencial Range Search TEST")
for i in radios:
    print("Testing with r =", i)
    for n in N_values:
        print("N =",n)
        start_time = time.time()
        data = dataset[:n]
        knn_Search = range_search(faces_encoding, data, i)
        print(knn_Search)
        end_time = time.time()
        tiempo_ejecucion = end_time - start_time
        print("El tiempo de exec es: ", tiempo_ejecucion)
    print("\n\n")

KNN-Secuencial Range Search TEST
Testing with r = 0.709143092728737
N = 100
['Angel_Maza_0001.jpg', 'Anthony_Pico_0001.jpg', 'Gro_Harlem_Brundtland_0001.jpg', 'Reginald_Hudlin_0001.jpg', 'Mike_Smith_0001.jpg', 'Xiang_Huaicheng_0001.jpg', 'Dragan_Covic_0001.jpg', 'Ranil_Wickremasinghe_0001.jpg']
El tiempo de exec es:  0.013332605361938477
N = 200
['Wilbert_Elki_Meza_Majino_0001.jpg', 'Jeong_Se-hyun_0001.jpg', 'Jeong_Se-hyun_0004.jpg', 'Jeong_Se-hyun_0009.jpg', 'Jeong_Se-hyun_0003.jpg', 'Jeong_Se-hyun_0006.jpg', 'Angel_Maza_0001.jpg', 'Jeong_Se-hyun_0008.jpg', 'Jeong_Se-hyun_0005.jpg', 'Jeong_Se-hyun_0002.jpg', 'Anthony_Pico_0001.jpg', 'Jeong_Se-hyun_0007.jpg', 'Gro_Harlem_Brundtland_0001.jpg', 'Ziwang_Xu_0001.jpg', 'Reginald_Hudlin_0001.jpg', 'Mike_Smith_0001.jpg', 'Xiang_Huaicheng_0001.jpg', 'Dragan_Covic_0001.jpg', 'Bill_Byrne_0001.jpg', 'Ranil_Wickremasinghe_0001.jpg', 'Jose_Alencar_0001.jpg', 'Gustavo_Cisneros_0001.jpg', 'Valorie_Brabazon_0001.jpg']
El tiempo de exec es:  0.01438236

### RTree

In [41]:
''' Testeando KNN-RTree '''
print("KNN-Rtree TEST")
for n in N_values:
    print("N =",n)
    start_time = time.time()
    print(knn_rtree(faces_encoding, k, dataset[:n]))
    end_time = time.time()
    tiempo_ejecucion = end_time - start_time
    print(tiempo_ejecucion)

KNN-Rtree TEST
N = 100
['/Users/ValDLaw/Documents/GitHub/2023-1/BDD2/Proyecto3/Proyecto3BDD2/backend/../images/Claudia_Pechstein/Claudia_Pechstein_0003.jpg', '/Users/ValDLaw/Documents/GitHub/2023-1/BDD2/Proyecto3/Proyecto3BDD2/backend/../images/Mike_Smith/Mike_Smith_0001.jpg', '/Users/ValDLaw/Documents/GitHub/2023-1/BDD2/Proyecto3/Proyecto3BDD2/backend/../images/Gro_Harlem_Brundtland/Gro_Harlem_Brundtland_0001.jpg', '/Users/ValDLaw/Documents/GitHub/2023-1/BDD2/Proyecto3/Proyecto3BDD2/backend/../images/Paul_ONeill/Paul_ONeill_0008.jpg', '/Users/ValDLaw/Documents/GitHub/2023-1/BDD2/Proyecto3/Proyecto3BDD2/backend/../images/Ranil_Wickremasinghe/Ranil_Wickremasinghe_0003.jpg', '/Users/ValDLaw/Documents/GitHub/2023-1/BDD2/Proyecto3/Proyecto3BDD2/backend/../images/German_Khan/German_Khan_0001.jpg', '/Users/ValDLaw/Documents/GitHub/2023-1/BDD2/Proyecto3/Proyecto3BDD2/backend/../images/Angel_Maza/Angel_Maza_0001.jpg', '/Users/ValDLaw/Documents/GitHub/2023-1/BDD2/Proyecto3/Proyecto3BDD2/backend